## Libraries installation

In [ ]:
!pip install tensorflow_hub
!pip install keras tf-models-official pydot graphviz
!pip install bert-tensorflow
!pip install transformers
!pip install tf-models-nightly
!pip install tensorflow-text-nightly
!pip install tf-models-official==2.5.0
!pip install tensorflow-text==2.5.0

!pip install tweepy

!pip install flask-ngrok
!pip install flask-bootstrap
!pip install pyngrok==4.1.1

     |████████████████████████████████| 2.2 MB 2.5 MB/s 
     |████████████████████████████████| 99 kB 7.7 MB/s 
     |████████████████████████████████| 1.1 MB 16.9 MB/s 
     |████████████████████████████████| 352 kB 43.8 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 47.8 MB 56 kB/s 
     |████████████████████████████████| 1.2 MB 35.8 MB/s 
     |████████████████████████████████| 636 kB 47.5 MB/s 
     |████████████████████████████████| 4.9 MB 31.2 MB/s 
     |████████████████████████████████| 237 kB 49.0 MB/s 
     |████████████████████████████████| 90 kB 8.2 MB/s 
     |████████████████████████████████| 462 kB 42.6 MB/s 
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl size=22257 sha256=17a02eedb6ef3589edb870f038174c19e67c0688cf1aabcc16c630d8e8925922
  Stored in directory: /root/.cache/pip/wheels/d2/f1/1f/041add21dc9c4220157f1bd2bd6afe1f1a49524c3396b94401
  Created wheel for seqeval: filename=seqeval-1

     |████████████████████████████████| 456 kB 6.7 MB/s 
  Created wheel for flask-bootstrap: filename=Flask_Bootstrap-3.3.7.1-py3-none-any.whl size=460123 sha256=1e13930442b44c717534cd3029b70331a5e74862ba07178f64fc5fc2ce5d288f
  Stored in directory: /root/.cache/pip/wheels/67/a2/d6/50d039c9b59b4caca6d7b53839c8100354a52ab7553d2456eb
  Created wheel for visitor: filename=visitor-0.1.3-py3-none-any.whl size=3946 sha256=2480a6790c9c6f100043c91f32e04957f847e9f608c9ddaad2bc3cf3181525b5
  Stored in directory: /root/.cache/pip/wheels/64/34/11/053f47218984c9a31a00f911ed98dda036b867481dcc527a12
Successfully built flask-bootstrap visitor
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15983 sha256=00e7e063b056f1ad568cdb68c785babb7b5e26cb4f629459a6be6fdf8c54ffee
  Stored in directory: /root/.cache/pip/wheels/b1/d9/12/045a042fee3127dc40ba6f5df2798aa2df38c414bf533ca765
Successfully built pyngrok


## Model Architecture 

In [ ]:
import os

import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub

from keras.utils import np_utils

import official.nlp.bert.bert_models
import official.nlp.bert.configs
import official.nlp.bert.run_classifier
import official.nlp.bert.tokenization as tokenization

from official.modeling import tf_utils
from official import nlp
from official.nlp import bert
import tweepy


gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  2.5.3
Eager mode:  True
Hub version:  0.12.0
GPU is NOT AVAILABLE


/usr/local/lib/python3.7/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.5.3 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


In [ ]:
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/2",
                            trainable=True)

In [ ]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [ ]:
def encode_names(n, tokenizer):
   tokens = list(tokenizer.tokenize(n))
   tokens.append('[SEP]')
   return tokenizer.convert_tokens_to_ids(tokens)

def bert_encode(string_list, tokenizer, max_seq_length):
  num_examples = len(string_list)
  
  string_tokens = tf.ragged.constant([
      encode_names(n, tokenizer) for n in np.array(string_list)])

  cls = [tokenizer.convert_tokens_to_ids(['[CLS]'])]*string_tokens.shape[0]
  input_word_ids = tf.concat([cls, string_tokens], axis=-1)

  input_mask = tf.ones_like(input_word_ids).to_tensor(shape=(None, max_seq_length))

  type_cls = tf.zeros_like(cls)
  type_tokens = tf.ones_like(string_tokens)
  input_type_ids = tf.concat(
      [type_cls, type_tokens], axis=-1).to_tensor(shape=(None, max_seq_length))

  inputs = {
      'input_word_ids': input_word_ids.to_tensor(shape=(None, max_seq_length)),
      'input_mask': input_mask,
      'input_type_ids': input_type_ids
      }

  return inputs

## Load model 

In [ ]:
#Importing data from google drive file
from google.colab import drive ##
drive.mount('/content/gdrive') ##

Mounted at /content/gdrive


In [ ]:
model_name = 'fake_news_model_FakeNet'
path = F"/content/gdrive/My Drive/Group_Project_530/Models/{model_name}" 
new_model = tf.keras.models.load_model(path, compile=False)

In [ ]:
text = ['indiafightscorona  deaths have been registered in the past  hours  of these are from  statesuts '] #Right fake
encoded_text = bert_encode(np.array(text), tokenizer, 100)
val = new_model.predict(encoded_text)
np.argmax(val)

0

## Front End Architecture

### Main functions

In [ ]:
consumer_key =  #ENTER KEY HERE AS A STRING 
consumer_secret =  #ENTER KEY HERE AS A STRING 
access_token =  #ENTER KEY HERE AS A STRING 
access_token_secret =  #ENTER KEY HERE AS A STRING 

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [ ]:
def tweet_prediction(tweet,model=new_model):
  encodedVal = bert_encode(np.array([tweet]), tokenizer, 100)
  pred_arr = model.predict(encodedVal)
  pred_val = np.argmax(pred_arr)
  if pred_val==1:
    return 'Fake'
  else:
    return 'Real'

In [ ]:
def superSpreaderCheck(tweet_df, user_df):

  value_counts_dict = dict(tweet_df.groupby('Label')['Label'].count())

  if 'Fake' in value_counts_dict.keys():
    fake_count= value_counts_dict['Fake']
  else:
    fake_count= 0

  size= int(len(tweet_df)/2)
  Influence_Score = float(user_df.Influence_Score)
  
  if(fake_count>=size and Influence_Score>=0.5):
    return True
  else:
    return False

In [ ]:
def get_user_data(user_ID, countOfTweets= 10):

  user_name = []
  num_followers = []
  num_following = []
  num_favorites= []
  num_retweets= []
  tweets_lst= []   

  try:
    tweets = api.user_timeline(screen_name= user_ID,count = countOfTweets)
  except tweepy.TweepError:              
    pass

  try:
    user_name_val= tweets[0].user.name
    user_name.append(user_name_val)

    user_followers= tweets[0].user.followers_count
    num_followers.append(user_followers) 

    user_friends= tweets[0].user.friends_count
    num_following.append(user_friends) 

  except tweepy.TweepError:              
        pass

  for tw in tweets:  
    try:
      tweets_new = api.get_status(tw.id)

      tweet_val = tweets_new.text
      tweets_lst.append(tweet_val)     

      user_likes = tweets_new.favorite_count
      num_favorites.append(user_likes) 

      user_retweets= tweets_new.retweet_count
      num_retweets.append(user_retweets) 

    except tweepy.TweepError:              
        pass

  userData = {
    'User_Name': user_name,  
    'NofFollowers': num_followers,
    'NofFollowing': num_following  
  }

  user_df= pd.DataFrame(userData)

  tweetData= {
    'Tweets': tweets_lst,
    'NofFavorites':  num_favorites,
    'NofRetweets': num_retweets
  }

  tweet_df= pd.DataFrame(tweetData)
  
  tweet_df['Label'] = tweet_df.Tweets.apply(lambda x: tweet_prediction(tweet=x))

  user_df["F2F_Ratio"] = user_df.NofFollowers/(user_df.NofFollowers+user_df.NofFollowing)

  user_df["Retweet_Ratio"] = sum(tweet_df.NofRetweets)/(sum(tweet_df.NofRetweets)+countOfTweets)

  user_df["Favorites_Ratio"] = sum(tweet_df.NofFavorites)/(sum(tweet_df.NofFavorites)+countOfTweets)

  user_df["Influence_Score"] = (user_df.F2F_Ratio + user_df.Retweet_Ratio + user_df.Favorites_Ratio)/3

  superSpreader = superSpreaderCheck(tweet_df, user_df)
  
  return superSpreader

### Flask settings

In [ ]:
cd /content/gdrive/My Drive/Group_Project_530

/content/gdrive/My Drive/Group_Project_530


In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template , request 
from pyngrok import ngrok
import os


ngrok_token =  #ENTER KEY HERE AS A STRING 

ngrok.set_auth_token(ngrok_token)
app = Flask(__name__)
run_with_ngrok(app)

@app.route('/')
def home():
    return render_template('index.html')


@app.route('/getprediction',methods=['POST'])
def getprediction():  
  input_lst = [str(x) for x in request.form.values()] 
  input_type = input_lst[1]
  twt_usr = input_lst[0]

  if input_type== 'tweet':
    encodedVal = bert_encode(np.array([twt_usr]), tokenizer, 100)
    pred_arr = new_model.predict(encodedVal)
    pred_val = np.argmax(pred_arr)
    if pred_val==1:
      prediction= 'Fake'
    else:
      prediction= 'Real'

    return render_template('index.html', output='Predicted tweet: {}'.format(prediction))
  
  elif input_type== 'user':

    superSpreader = get_user_data(twt_usr, countOfTweets= 20)

    if superSpreader:
      prediction= 'Superspreader!'
    else:
      prediction= 'not a Superspreader!'
    
    return render_template('index.html', output='The user is {}'.format(prediction)) 

if __name__ == '__main__':
   app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://a85b-34-125-187-20.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [12/May/2022 22:27:55] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/May/2022 22:27:56] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [12/May/2022 22:28:22] "POST /getprediction HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/May/2022 22:29:07] "POST /getprediction HTTP/1.1" 200 -
